In [1]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import geopandas as gpd
import rasterio
import pandas as pd
from functools import reduce

from sovereign.utils import (map_flopros_to_adm, union_gdfs, calculate_river_length_per_admin,
                            calculate_increased_protection, calculate_increased_protection_costs)

#### Set filepaths and provide data info

In [2]:
root = Path.cwd().parent # find project root
flopros_path = os.path.join(root, 'inputs', 'flood', 'protection', 'flopros-THA.shp')
adm_path = os.path.join(root, 'inputs', 'boundaries', 'admin', 'gadm36_THA_1.shp')
mapping_csv = os.path.join(root, 'inputs', 'flood', 'protection', 'flopros-adm1-map.csv')
basin_path = os.path.join(root, 'inputs', 'boundaries', 'basins', 'BA_THA_lev06.shp')
output_path = os.path.join(root, 'outputs', 'boundaries', 'analysis_basins.gpkg')
river_path = os.path.join(root, 'inputs', 'flood', 'rivers', 'hydroRIVERS_v10_THA.shp')
urban_path = os.path.join(root, 'inputs', 'exposure', 'ghsl_du', 'ghsl_du_gadm_THA.gpkg')

In [3]:
# Load data
flopros = gpd.read_file(flopros_path)
adm = gpd.read_file(adm_path)
mapping_df = pd.read_csv(mapping_csv)
basins = gpd.read_file(basin_path)
rivers = gpd.read_file(river_path)
urban = gpd.read_file(urban_path)

#### Baseline Flood Protection

In [4]:
# Map protection levels to the Admin Dataset
protection_levels = map_flopros_to_adm(mapping_df, flopros, adm)

In [5]:
# Merge protection levels with GADM dataset
# Rename the columns we want to merge
basins = basins.rename(columns={'HYBAS_ID': 'HYBAS_ID_06'})
protection_levels = protection_levels.rename(columns={'GID_1': 'flpr_gid_1'})
# Select relevant columns 
basins_selected = basins[['geometry', 'HYBAS_ID_06']]
protection_levels_selected = protection_levels[['geometry', 'flpr_gid_1', 'NAME', 'MerL_Riv']]
datasets = [basins_selected, protection_levels_selected]
merged = reduce(union_gdfs, datasets)

e:\projects\sovereign-risk\sovereign\utils.py:207: UserWarning: `keep_geom_type=True` in overlay resulted in 224 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  return gpd.overlay(gdf1, gdf2_copy, how='union')


In [6]:
# Clean dataset (remove all rows where there are no column values consistently accross the rows)
columns_to_check = ["HYBAS_ID_06", "flpr_gid_1"]
# Create a boolean mask for non-null and non-zero values
mask = (merged[columns_to_check].notnull() & (merged[columns_to_check] != 0)).all(axis=1)
# Filter the GeoDataFrame to keep only rows that meet the criteria
filtered_merged = merged.loc[mask]

In [7]:
# Save to file
filtered_merged.to_file(output_path)

#### Adaptation Scenario

In [8]:
# User Config
min_urban = 23 # towns and cities see -> https://human-settlement.emergency.copernicus.eu/documents/GHSL_Data_Package_2023.pdf?t=1727170839#page=66.08
adaptation_target = 100 # return period
river_size_limit = 500 # upstream catchment area (km^2)
adaptation_unit_cost = 2399000 # $2.399 million per km unit cost from Boulange paper

In [9]:
# Calculate river length within each basin
adaptation = calculate_river_length_per_admin(protection_levels, rivers, river_size_limit, urban, min_urban)
# Calculate how much additional protection is needed to reach a target protection level
adaptation = calculate_increased_protection(adaptation, adaptation_target)
# Calculate how much this additional protection will cost (using Boulange et al 2023 method)
adaptation = calculate_increased_protection_costs(adaptation, adaptation_unit_cost)

In [10]:
# Create a raster mask with areas to be protected 
output_path = os.path.join(root, 'outputs', 'flood', 'adaptation', f'urban_mask_{min_urban}.tif')
ref_raster_path = os.path.join(root, 'inputs', 'flood', 'maps', 'THA_jrc-flood_RP10.tif') # referance raster for creation
filtered_urbanisation = urban[urban['L2'] >= min_urban]

# Open the reference raster to use its dimensions and CRS
with rasterio.open(ref_raster_path) as ref:
    # Create an empty mask with the same dimensions as the reference raster
    mask = rasterio.features.rasterize(
        ((geom, 1) for geom in filtered_urbanisation.geometry),
        out_shape=ref.shape,
        transform=ref.transform,
        fill=0,  # Fill value for 'background'
        all_touched=False,  # Only if centroids touch. 
        dtype='uint8'
    )

# Save the mask raster
with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=ref.height,
    width=ref.width,
    count=1,
    dtype='uint8',
    crs=ref.crs,
    transform=ref.transform,
) as dst:
    dst.write(mask, 1)

In [11]:
print(f'Adapting Urban DUC{min_urban} and denser regions to RP{adaptation_target} costs:', adaptation['Add_Pr_c_u'].sum())

Adapting Urban DUC23 and denser regions to RP100 costs: 21973517577.964287
